In [3]:
import numpy as np
import os 
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from ultralytics import YOLO
from ultralytics.utils.ops import non_max_suppression

In [4]:


class NumpyArrayDataset(Dataset):
    def __init__(self, np_arrays , target_shape = (640 , 640)):
        self.data = np_arrays
        self.target_shape = target_shape  # shape: (H, W) or (H, W)

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        arr = self.data[idx]
        arr = np.transpose(arr, (2, 0, 1))
        arr = torch.from_numpy(arr).float() / 255.0  # Normalize to [0, 1] range

        arr = F.interpolate(arr.unsqueeze(0), size=self.target_shape, mode='bilinear', align_corners=False).squeeze(0)
        return arr

    @staticmethod
    def from_folder(folder_path : str) -> np.ndarray:

        rgb_array = []

        for filename in os.listdir(folder_path):
            if filename.endswith('.npy') and filename.startswith('rgb'):
                file_path = os.path.join(folder_path, filename)
                data = np.load(file_path, allow_pickle=True)
                rgb_array.append(data)

        data = np.array(rgb_array)

        return data
            

# Usage:

folder_path = 'run1_data/home/wasteflow/Desktop/live_tests/run1/data'
loaded_data = NumpyArrayDataset.from_folder(folder_path)
dataset = NumpyArrayDataset(loaded_data)
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

In [10]:

model = YOLO('weights/yolov8n.pt')

for batch in loader:
    

    
    result = model.model(batch)[0]
    bbox = result[:, :4, :]  # Bounding box coordinates

    result_sigmoid = torch.sigmoid(result[:, 4:14, :])

    combined = torch.cat((bbox, result_sigmoid), dim=1)

    nms = non_max_suppression(combined , conf_thres=0.25, iou_thres=0.45, max_det=100)

    
    print(nms[0][:, -1])  # Print the first 6 columns of the first detection



tensor([3.])
tensor([1.])
tensor([1.])
tensor([3., 1.])
tensor([1., 1., 1., 8.])
tensor([3., 1., 1.])
tensor([1.])
tensor([])
tensor([6.])
tensor([1.])
tensor([1., 1., 1., 1.])
tensor([3.])
tensor([])
tensor([1., 1., 1.])
tensor([1., 1.])
tensor([3., 1.])


KeyboardInterrupt: 

In [16]:
a = next(iter(loader)) 

model.model(a)[0][0][4:14, :]

tensor([[-12.8202, -13.6976, -13.1619,  ..., -12.7952, -14.2141, -14.6250],
        [-10.8883, -11.8876, -11.4052,  ..., -11.7456, -13.6239, -13.8637],
        [-14.3954, -14.9226, -14.5575,  ..., -12.8190, -14.4662, -15.1281],
        ...,
        [-14.3390, -14.7718, -14.5611,  ..., -12.4586, -13.9337, -14.4225],
        [-12.8931, -13.6733, -13.1546,  ..., -11.8407, -13.3013, -13.8620],
        [-11.5978, -11.9107, -11.1452,  ..., -12.1831, -13.8952, -14.4330]])